In [ ]:
# path to save file
path = r'D:\\Data Science\\ADM(Algorithmic Methods of Data Mining)\\hw\\hw4\\'

# the function will create the data and return it, just give the path to save the file

def create_data(path):
    # Create a dataframe to store all data

    # find max page number
    max_page_no = int(BeautifulSoup(requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1").text,
                                    "lxml").find_all('span',attrs={"class": "pagination__label"})[-1:][0].text)

    # save the file after each iteration with pickle 
    fileObject = open(path + "adm-hw4-data-dene",'wb')

    for page in range(1,max_page_no+1): # go through for all pages

        try:
            url = "https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=" + str(page)

            print("page:",page)

            content = requests.get(url) # create a content for specified url
            soup  = BeautifulSoup(content.text, "lxml") # turn it as lxml format

            all_adv = soup.find_all('p',attrs={"class": "titolo text-primary"})

            # store the data for only 1 page in a dataframe
            data = pd.DataFrame(columns=["url","price", "locali", "superficie", "bagni", "piano", "description"])

            for link_i in range(len(all_adv)):
                link = all_adv[link_i].find('a').get('href') # get all link that are advertised on the page that we specified above

                if 'https://' in link:
                    linkURL = link
                else: # some of link do not have http, http://, www.immobiliare.it or both parts. Thus, add it at the beginning of link
                    if 'www.immobiliare.it' in link:
                        if link[:2] == '//':
                            linkURL = 'https:' + link
                        elif link[0] == '/':
                            linkURL = 'https:/' + link
                    else:
                        if link[0] != '/':
                            linkURL = 'https://www.immobiliare.it/' + link
                        else:
                            linkURL = 'https://www.immobiliare.it' + link

                print(linkURL)

                link_content = requests.get(linkURL)
                link_soup = BeautifulSoup(link_content.text, "lxml") # convert the content into lxml
                ul = link_soup.find_all("ul", attrs={"class": "list-inline list-piped features__list"}) # this list includes all features except price

                # check which features having and store it if there is present or not
                features = [] # at the end, it'll show that locali, superficie, bagni and piano exist in the link or not
                all_features = list(map(lambda x: x.text, ul[0].find_all('div',attrs={"class": "features__label"}))) # which featers link have
                features_check_list =['locali', 'superficie', 'bagni', 'piano']

                for i in range(len(features_check_list)):
                    if features_check_list[i] in all_features:
                        features.append(1) # 1 means that feature is present
                    else:
                        features.append(0) # 0 means that feature is not present


                feature_values = [] # all features will be on that list   


                # first add linkURL
                feature_values.append(linkURL)


                # add avg. price to feature_values
                price_block = link_soup.find_all('ul',attrs={"class": "list-inline features__price-block"})[0].find_all('li',attrs={"class": "features__price"})[0]
                price = []

                if not(str(str(price_block).find("features__price--double")).isdigit()) and not(str(str(price_block).find("features__price-old")).isdigit()):
                    for s in price_block.text.split():
                        if s.isdigit() or s.replace('.','').isdigit(): # check whether it is int or float
                            s = s.replace('.','') # web site uses dot instead of comma. So first destroy dots
                            s = s.replace(',','.') # then replace comma with dot because in python decimal numbers indicates w/ dot
                            price.append(float(s))
                elif str(str(price_block).find("features__price--double")).isdigit():
                    # for the price feature, sometimes a range is given. In that case, we'll take average of min and max value of price
                    for s in price_block.text.split():
                        if s.isdigit() or s.replace('.','').isdigit(): # check whether it is int or float
                            s = s.replace('.','')
                            s = s.replace(',','.')
                            price.append(float(s))
                elif str(str(price_block).find("features__price-old")).isdigit():
                    start_idx = str(price_block).find('<li class="features__price"><span>') + len('<li class="features__price"><span>')
                    end_idx = str(price_block).find("</span>") 
                    for s in str(price_block)[start_idx:end_idx].split():
                        if s.isdigit() or s.replace('.','').isdigit(): # check whether it is int or float
                            s = s.replace('.','')
                            s = s.replace(',','.')
                            price.append(float(s))

                feature_values.append(np.mean(price))


                # fill the features; locali, superficie, bagni and piano (price is already added.)
                loc_sficie_bag = list(map(lambda x: x.text, ul[0].find_all('span',attrs={"class": "text-bold"})))

                j = 0
                for i in range(3): # we'll fill locali, superficie and bagni
                    if features[i] == 0: # we are checking absence of the feature
                        feature_values.append(None) # if it is absent, put it None 
                    else:
                        if i == 0:
                            # this part is only for locali. If there is range for locali, take it average ot it
                            loc = []
                            for e in loc_sficie_bag[j]:
                                for s in e.split():
                                    if s.isdigit() or s.replace('.','',1).isdigit(): # check whether it is int or float
                                        loc.append(float(s))
                            feature_values.append(np.mean(loc)) # take it average and add the value to feature_values
                            j += 1
                        else:
                            feature_values.append(int(re.search(r'\d+', loc_sficie_bag[j]).group())); j += 1


                # adding piano; it can be integer or string
                piano = ul[0].find_all('abbr',attrs={"class": "text-bold im-abbr"})

                if piano != []: # check whether piano feature does not exist in the link or not
                    feature_values.append(piano[0].text.split("\xa0")[0]) # if it exists, add the value to feature_values 
                else:
                    feature_values.append(None) # if it does not exists, add None to feature_values


                # adding description
                desc = link_soup.find_all('div',attrs={"id": "description"})[0].find_all('div',attrs={"class": "col-xs-12 description-text text-compressed"})[0].text
                feature_values.append(desc)

                data.loc[data.shape[0]+1]= feature_values # add all features as new row

                time.sleep(0.5)

            pickle.dump(data, fileObject) # save the dataframe that we got for just 1 page
            time.sleep(0.5) # this helps to prevent the website block
        except:
            pass


    fileObject.close()
    
    # read the data part by part
    ADM_HW4_data = pd.DataFrame(columns=["url","price", "locali", "superficie", "bagni", "piano", "description"]) # rename columns

    fileObject = open(path + "adm-hw4-data-dene",'rb') # open the file to read

    # to not force memory, we send the data for each page to pickle. Now, we are collecting them. Since we use try and except,
    # some of pages are lost (we have 1729 at the beginning) but around 41000 rows are quite enough also.
    for i in range(1,1678+1):
        ADM_HW4_data = ADM_HW4_data.append(pickle.load(fileObject))

    fileObject.close() # close to file

    ADM_HW4_data.reset_index(drop=True, inplace=True) # drop indexes

    # since we create data from too many pickle files, I will save it as one piece
    ADM_HW4_data.to_pickle(path + 'hw4_data')

    # read the data
    hw4_data = pd.read_pickle(path + 'hw4_data')
    
    return hw4_data